# Apply Random Forest

In [1]:
library(ggplot2)
library(caret)
library(e1071)
library(dplyr)
library(randomForest)

Loading required package: lattice

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
# 利用する特徴量を列挙
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
# 学習誤差の表を作成する
trainAndPredict <- function(dfx, verbose=FALSE) {        
    train <- dfx[, c(valuables, "flag")]       # AccelOrBrake を除外
    test <- dfx
    fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=500)
    p <- predict(fit, newdata=test)

    # 表を作成するのにもととなるものを出力
    showMatrix(test, p)
}

In [6]:
showMatrix <- function(test, predictedFlags) {
    # test のうち、Red, Blue と判断したものを取得
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    
    # これらのうち、Red or Blue, Accel or Brake でグループ分けし、それらの個数を出力
    printf("Red\n")
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    printf("Blue\n")
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    printf("\n")
}

# Predict Red

In [7]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [8]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [9]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [10]:
fomula <- paste("flag ~ ", paste(valuables, collapse=" + "))

In [11]:
set.seed(1)

In [12]:
# 使用する特徴量だけ抜き取る
df <- df3[, c(valuables, "AccelOrBrake", "flag")]

In [13]:
trainAndPredict(df, verbose=TRUE)

Red
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1    Red        Accel   401
2    Red        Brake   192
Blue
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    77
2   Blue        Brake   372

